In [ ]:
model = models.resnet152(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth
100%|██████████| 230M/230M [00:03<00:00, 69.9MB/s]


In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import torchvision.models as models
import os
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

In [ ]:
features_extractor = torch.nn.Sequential(*list(model.children())[:-1])

In [ ]:
preprocess =transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [ ]:
files=os.listdir('/content/drive/MyDrive/archive/Covid19-dataset/train/Covid')

In [ ]:
def resnet_features(path):
  lst=[]
  files=os.listdir(path)
  for file in files:
    image = Image.open(path+'/'+file).convert('RGB')
    image=preprocess(image)
    image = image.unsqueeze(0)
    features = features_extractor(image)
    features = torch.flatten(features, start_dim=1)
    features_vector = features.detach().numpy().squeeze()
    features_vector=list(features_vector)
    lst.append(np.array(features_vector))
  return lst

In [ ]:
train_covid_features=resnet_features('/content/drive/MyDrive/archive/Covid19-dataset/train/Covid')
train_normal_features=resnet_features('/content/drive/MyDrive/archive/Covid19-dataset/train/Normal')
train_pneumonia_features=resnet_features('/content/drive/MyDrive/archive/Covid19-dataset/train/Viral_Pneumonia')


In [ ]:
test_covid_features=resnet_features('/content/drive/MyDrive/archive/Covid19-dataset/test/Covid')
test_normal_features=resnet_features('/content/drive/MyDrive/archive/Covid19-dataset/test/Normal')
test_pneumonia_features=resnet_features('/content/drive/MyDrive/archive/Covid19-dataset/test/Viral_Pneumonia')

In [ ]:
train_data_features=train_normal_features+train_covid_features+train_pneumonia_features
train_abnormal=[0]*len(train_normal_features)+[1]*(len(train_data_features)-len(train_normal_features))
train_disease=[0]*len(train_normal_features)+[1]*(len(train_covid_features))+[2]*(len(train_pneumonia_features))
train_data={"image_features":train_data_features,'Abnormal':train_abnormal,'Disease':train_disease}
train_data = pd.DataFrame(train_data)

In [ ]:
test_data_features=test_normal_features+test_covid_features+test_pneumonia_features
test_abnormal=[0]*len(test_normal_features)+[1]*(len(test_data_features)-len(test_normal_features))
test_disease=[0]*len(test_normal_features)+[1]*(len(test_covid_features))+[2]*(len(test_pneumonia_features))
test_data={"image_features":test_data_features,'Abnormal':test_abnormal,'Disease':test_disease}
test_data = pd.DataFrame(test_data)


In [ ]:
train_data = shuffle(train_data)
test_data = shuffle(test_data)

In [ ]:
train_data.head()

,image_features,Abnormal,Disease
47,"[0.7761361, 0.53914136, 0.85161215, 0.47204667...",0,0
159,"[0.78768665, 0.4862475, 0.8897891, 0.44288307,...",1,1
37,"[0.79049295, 0.45279664, 0.79018575, 0.4751636...",0,0
216,"[0.7809896, 0.5364811, 0.8907847, 0.46811676, ...",1,2
122,"[0.865444, 0.49583006, 0.9519542, 0.50982296, ...",1,1


In [ ]:
test_data.head()

,image_features,Abnormal,Disease
51,"[0.84016514, 0.5066724, 0.925868, 0.43348998, ...",1,2
14,"[0.75564116, 0.5191521, 0.9100316, 0.5163007, ...",0,0
49,"[0.82369083, 0.49209556, 0.8964677, 0.40374428...",1,2
16,"[0.7608028, 0.5138447, 0.91849166, 0.48358038,...",0,0
38,"[0.8308714, 0.4939494, 0.8535036, 0.47717226, ...",1,1


In [ ]:
train_features = np.array(train_data['image_features'].tolist())
train_features = train_features / 255.0
train_abnormal_labels = np.array(train_data['Abnormal'])
train_disease_labels = np.array(train_data['Disease'])

In [ ]:
test_features = np.array(test_data['image_features'].tolist())
test_features = test_features / 255.0
test_abnormal_labels = np.array(test_data['Abnormal'])
test_disease_labels = np.array(test_data['Disease'])

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

# Assuming train_features, train_abnormal_labels, train_disease_labels,
# test_features, test_abnormal_labels, and test_disease_labels are already defined

# Define the first model to classify normal vs. abnormal
model_normal_abnormal = Sequential()
model_normal_abnormal.add(Dense(128, input_dim=train_features.shape[1], activation='relu'))
model_normal_abnormal.add(Dense(1, activation='sigmoid'))

# Compile the model for binary classification (normal vs. abnormal)
model_normal_abnormal.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping for the first model based on validation loss
early_stopping_normal_abnormal = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the first model with early stopping and validation data
model_normal_abnormal.fit(train_features, train_abnormal_labels, epochs=100, batch_size=32,
                         validation_data=(test_features, test_abnormal_labels), callbacks=[early_stopping_normal_abnormal])

# Predict on test data for binary classification
abnormal_predictions = model_normal_abnormal.predict(test_features)

# Separate abnormal images for further classification
abnormal_indices = np.where(abnormal_predictions > 0.5)[0]

if len(abnormal_indices) > 0:
    abnormal_features = test_features[abnormal_indices]
    abnormal_labels = test_disease_labels[abnormal_indices]

    # Define the second model to classify COVID vs. pneumonia for abnormal images
    model_covid_pneumonia = Sequential()
    model_covid_pneumonia.add(Dense(128, input_dim=abnormal_features.shape[1], activation='relu'))
    model_covid_pneumonia.add(Dense(3, activation='softmax'))

    # Compile the model for multiclass classification (COVID vs. pneumonia vs. no disease)
    model_covid_pneumonia.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Define early stopping for the second model based on validation loss
    early_stopping_covid_pneumonia = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the second model with early stopping and validation data
    model_covid_pneumonia.fit(abnormal_features, abnormal_labels, epochs=100, batch_size=32,
                              validation_data=(test_features[abnormal_indices], test_disease_labels[abnormal_indices]),
                              callbacks=[early_stopping_covid_pneumonia])

    # Evaluate the first model on the test set
    normal_abnormal_test_loss, normal_abnormal_test_acc = model_normal_abnormal.evaluate(test_features, test_abnormal_labels)

    # Evaluate the second model on the abnormal images from the test set
    covid_pneumonia_test_loss, covid_pneumonia_test_acc = model_covid_pneumonia.evaluate(test_features[abnormal_indices], test_disease_labels[abnormal_indices])

    print(f"Normal vs. abnormal test accuracy: {normal_abnormal_test_acc}")
    print(f"COVID vs. pneumonia test accuracy: {covid_pneumonia_test_acc}")
else:
    print("No abnormal predictions were made by the first model.")


Epoch 1/100
8/8 [==============================] - 5s 33ms/step - loss: 0.6757 - accuracy: 0.6494 - val_loss: 0.6615 - val_accuracy: 0.6970
Epoch 2/100
8/8 [==============================] - 0s 10ms/step - loss: 0.6442 - accuracy: 0.7211 - val_loss: 0.6375 - val_accuracy: 0.6970
Epoch 3/100
8/8 [==============================] - 0s 10ms/step - loss: 0.6225 - accuracy: 0.7211 - val_loss: 0.6212 - val_accuracy: 0.6970
Epoch 4/100
8/8 [==============================] - 0s 10ms/step - loss: 0.6035 - accuracy: 0.7211 - val_loss: 0.6147 - val_accuracy: 0.6970
Epoch 5/100
8/8 [==============================] - 0s 10ms/step - loss: 0.5954 - accuracy: 0.7211 - val_loss: 0.6136 - val_accuracy: 0.6970
Epoch 6/100
8/8 [==============================] - 0s 9ms/step - loss: 0.5936 - accuracy: 0.7211 - val_loss: 0.6151 - val_accuracy: 0.6970
Epoch 7/100
8/8 [==============================] - 0s 10ms/step - loss: 0.5920 - accuracy: 0.7211 - val_loss: 0.6158 - val_accuracy: 0.6970
Epoch 8/100
8/8 [====

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Predict using the first model
abnormal_predictions = model_normal_abnormal.predict(test_features)
abnormal_predictions = (abnormal_predictions > 0.5).astype(int)

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(test_abnormal_labels, abnormal_predictions)
precision = precision_score(test_abnormal_labels, abnormal_predictions)
recall = recall_score(test_abnormal_labels, abnormal_predictions)
f1 = f1_score(test_abnormal_labels, abnormal_predictions)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# Confusion matrix
confusion = confusion_matrix(test_abnormal_labels, abnormal_predictions)
print("Confusion Matrix:")
print(confusion)


3/3 [==============================] - 0s 5ms/step
Accuracy: 0.70
Precision: 0.70
Recall: 1.00
F1 Score: 0.82
Confusion Matrix:
[[ 0 20]
 [ 0 46]]
